In [6]:
import numpy as np
import pandas as pd
import os
import time
import threading
import tkinter as tk
import ta
from tkinter import ttk
from binance import Client


In [7]:
api_key = os.environ.get('API_KEY')
secret_key = os.environ.get('API_KEY_PRIVATE')

# Create connection to Binance
client = Client(api_key, secret_key)

In [8]:
#read BTCUSDT data file
df = pd.read_csv('data/BTCUSDT_data.csv')
#drop unamed and index columns
df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)

In [51]:
def use_macd(df):
    df['macd'], df['signal'], df['histogram'] = macd(df.Close)
    df['prev_histogram'] = df['histogram'].shift(1)
    # trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    df['macd_buy_sell'] = 0

    buy_condition_1 = (df['prev_histogram'] < 0) & (df['histogram'] > 0) 
    buy_condition_2 = (df['macd'] < 0) & (df['signal'] < 0)
    
    df.loc[buy_condition_1 & buy_condition_2, 'macd_buy_sell'] = 1

    sell_signal_1 = (df['prev_histogram'] > 0) & (df['histogram'] < 0) 
    sell_signal_2 = (df['macd'] > 0) & (df['signal'] > 0)

    df.loc[sell_signal_1 & sell_signal_2, 'macd_buy_sell'] = -1

    #Clean up df
    df.drop(['macd', 'signal', 'histogram', 'prev_histogram'], axis = 1, inplace = True)

    return df

In [52]:
# Testing macd_function
macd_test = use_macd(df)
macd_test[(macd_test['macd_buy_sell'] == 1) | (macd_test['macd_buy_sell'] == -1)]

,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell
54,BTCUSDT,2022-11-30 20:54:00,17044.56,17053.00,17043.05,17046.15,154.12992,1
68,BTCUSDT,2022-11-30 21:08:00,17074.85,17077.27,17054.49,17060.11,252.28819,-1
88,BTCUSDT,2022-11-30 21:28:00,17036.55,17049.21,17035.78,17047.69,152.91954,1
123,BTCUSDT,2022-11-30 22:03:00,17094.58,17100.41,17092.00,17098.90,134.64526,-1
151,BTCUSDT,2022-11-30 22:31:00,17102.90,17106.74,17096.12,17096.95,199.00515,-1
...,...,...,...,...,...,...,...,...
90888,BTCUSDT,2023-02-01 22:48:00,23712.21,23722.00,23700.01,23715.51,89.66899,1
90909,BTCUSDT,2023-02-01 23:09:00,23711.72,23716.91,23699.00,23714.58,244.42854,-1
90926,BTCUSDT,2023-02-01 23:26:00,23754.84,23755.81,23743.31,23744.74,125.11862,-1
90951,BTCUSDT,2023-02-01 23:51:00,23701.50,23714.42,23701.22,23708.88,126.76657,1


In [53]:
def use_sma(df):
    df['sma_50'] = sma(df.Close, 50)
    df['sma_200'] = sma(df.Close, 200)
    df['sma_diff'] = df['sma_50'] - df['sma_200']
    df['prev_sma_diff'] = df['sma_diff'].shift(1)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['sma_buy_sell'] = 0

    buy_condition = (trading_df['sma_diff'] > 0) & (trading_df['prev_sma_diff'] < 0)
    sell_condition = (trading_df['sma_diff'] < 0) & (trading_df['prev_sma_diff'] > 0)

    trading_df.loc[buy_condition, 'sma_buy_sel'] = 1
    trading_df.loc[sell_condition, 'sma_buy_sell'] = -1

    # Clean up df
    df.drop(['sma_50', 'sma_200', 'sma_diff', 'prev_sma_diff'], axis = 1, inplace = True)

    return trading_df

In [54]:
# Test sma
sma_test = use_sma(df)
sma_test[(sma_test['sma_buy_sell'] == 1) | (sma_test['sma_buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\1126140099.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['sma_buy_sell'] = 0
C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\1126140099.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df.loc[buy_condition, 'sma_buy_sel'] = 1


,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell,sma_50,sma_200,sma_diff,prev_sma_diff,sma_buy_sell,sma_buy_sel
329,BTCUSDT,2022-12-01 01:29:00,17122.48,17127.38,17119.81,17123.66,165.05873,0,17149.3450,17149.40965,-0.06465,0.89295,-1,NaN
575,BTCUSDT,2022-12-01 05:35:00,17110.78,17116.36,17104.74,17108.92,313.47280,0,17139.2864,17139.87275,-0.58635,0.08915,-1,NaN
824,BTCUSDT,2022-12-01 09:44:00,17105.88,17110.91,17100.91,17102.47,214.30144,0,17100.2578,17100.55335,-0.29555,0.10425,-1,NaN
903,BTCUSDT,2022-12-01 11:03:00,17063.53,17076.62,17060.45,17071.65,238.27665,0,17097.3612,17097.60200,-0.24080,0.18180,-1,NaN
1094,BTCUSDT,2022-12-01 14:14:00,17085.44,17086.93,17079.24,17081.82,133.85090,0,17108.0574,17108.27260,-0.21520,0.66850,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89312,BTCUSDT,2023-01-31 20:32:00,23095.23,23101.02,23075.35,23078.03,284.74586,0,23140.1902,23141.65685,-1.46665,0.09510,-1,NaN
89684,BTCUSDT,2023-02-01 02:44:00,23107.13,23107.13,23086.92,23089.15,224.67485,-1,23101.9860,23102.13515,-0.14915,0.40305,-1,NaN
89727,BTCUSDT,2023-02-01 03:27:00,23072.11,23074.46,23063.57,23072.42,127.62384,0,23094.4946,23094.88850,-0.39390,0.32845,-1,NaN
89902,BTCUSDT,2023-02-01 06:22:00,23088.80,23102.00,23082.32,23098.91,167.89105,0,23123.4918,23123.92500,-0.43320,0.61005,-1,NaN


In [55]:
def use_rsi(df, overbought_thresh = 70, oversold_thresh = 30):
    df['rsi'] = rsi(df.Close, 14)
    df['prev_rsi'] = df['rsi'].shift(1)

    # df['self_rsi'] = self_rsi(df.Close)

    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['rsi_buy_sell'] = 0

    buy_condition = (trading_df['rsi'] < oversold_thresh) & (trading_df['prev_rsi'] >= oversold_thresh)
    sell_condition = (trading_df['rsi'] > overbought_thresh) & (trading_df['prev_rsi'] <= overbought_thresh)

    trading_df.loc[buy_condition, 'rsi_buy_sell'] = 1
    trading_df.loc[sell_condition, 'rsi_buy_sell'] = -1
    # Clean up df
    trading_df.drop(['rsi', 'prev_rsi'], axis = 1, inplace = True)
    return trading_df

In [56]:
# Test rsi
rsi_test = use_rsi(df)
rsi_test[(rsi_test['rsi_buy_sell'] == 1) | (rsi_test['rsi_buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\2050756131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['rsi_buy_sell'] = 0
C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\2050756131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df.drop(['rsi', 'prev_rsi'], axis = 1, inplace = True)


,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell,rsi_buy_sell
442,BTCUSDT,2022-12-01 03:22:00,17157.35,17169.37,17155.50,17164.05,186.65042,0,-1
572,BTCUSDT,2022-12-01 05:32:00,17130.40,17132.07,17116.01,17116.34,347.21659,0,1
578,BTCUSDT,2022-12-01 05:38:00,17112.69,17113.43,17106.10,17108.22,128.66704,0,1
582,BTCUSDT,2022-12-01 05:42:00,17106.88,17110.61,17103.04,17103.24,98.48107,0,1
649,BTCUSDT,2022-12-01 06:49:00,17118.59,17125.78,17117.67,17123.31,116.30316,0,-1
...,...,...,...,...,...,...,...,...,...
90715,BTCUSDT,2023-02-01 19:55:00,23439.46,23479.48,23412.84,23457.48,630.94951,0,-1
90749,BTCUSDT,2023-02-01 20:29:00,23522.00,23554.81,23500.08,23553.13,654.17634,0,-1
90755,BTCUSDT,2023-02-01 20:35:00,23584.02,23629.00,23575.00,23625.60,346.55442,0,-1
90786,BTCUSDT,2023-02-01 21:06:00,23641.90,23704.98,23640.29,23664.09,1421.27784,0,-1


In [57]:
def use_ichimoku(df):
    df['conversion_line'], df['base_line'], df['leading_span_a'], df['leading_span_b'], df['lagging_span'] = ichimoku_cloud(df.High, df.Low, df.Close)
    df['conversion_base_diff'] = df['conversion_line'] - df['base_line']
    df['prev_diff'] = df['conversion_base_diff'].shift(1)
    
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['ichimoku_buy_sell'] = 0

    buy_condition_1 = trading_df.Close >= np.maximum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    buy_condition_2 = (trading_df['conversion_base_diff'] > 0) & (trading_df['prev_diff'] < 0)
    trading_df.loc[buy_condition_1 & buy_condition_2, 'ichimoku_buy_sell'] = 1

    sell_condition_1 = trading_df.Close < np.minimum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    sell_condition_2 = (trading_df['conversion_base_diff'] < 0) & (trading_df['prev_diff'] > 0)
    trading_df.loc[sell_condition_1 & sell_condition_2, 'ichimoku_buy_sell'] = -1

    # Clean up df
    df.drop(['conversion_line', 'base_line', 'leading_span_a', 'leading_span_b', 'lagging_span', 'conversion_base_diff', 'prev_diff'], axis = 1, inplace = True)
    return trading_df

In [58]:
#Test ichimoku
ichimoku_test = use_ichimoku(df)
ichimoku_test[(ichimoku_test['ichimoku_buy_sell'] == 1) | (ichimoku_test['ichimoku_buy_sell'] == -1)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\3658784860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['ichimoku_buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell,rsi,prev_rsi,conversion_line,base_line,leading_span_a,leading_span_b,lagging_span,conversion_base_diff,prev_diff,ichimoku_buy_sell
256,BTCUSDT,2022-12-01 00:16:00,17194.13,17220.00,17192.32,17217.68,509.72010,0,67.103912,59.743856,17186.855,17176.675,17192.7175,17168.605,17165.03,10.180,-9.490,1
315,BTCUSDT,2022-12-01 01:15:00,17145.44,17151.59,17144.92,17149.36,90.33405,1,47.362174,44.584934,17146.200,17146.485,17174.3250,17184.820,17129.16,-0.285,2.855,-1
320,BTCUSDT,2022-12-01 01:20:00,17142.10,17144.19,17135.84,17140.24,123.01189,0,41.788835,42.981114,17146.535,17152.195,17158.8025,17179.470,17134.29,-5.660,0.050,-1
354,BTCUSDT,2022-12-01 01:54:00,17129.61,17131.00,17122.94,17124.62,126.71598,0,43.072347,47.478344,17130.435,17130.920,17132.5125,17142.680,17121.70,-0.485,6.735,-1
381,BTCUSDT,2022-12-01 02:21:00,17122.44,17123.22,17109.00,17113.50,195.92541,0,35.912622,41.887798,17124.300,17125.995,17126.2650,17137.845,17129.72,-1.695,3.870,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90674,BTCUSDT,2023-02-01 19:14:00,23097.25,23100.00,23066.44,23082.05,272.82793,0,51.858301,54.268024,23059.000,23057.755,23056.0300,23033.915,23166.34,1.245,-2.810,1
90677,BTCUSDT,2023-02-01 19:17:00,23045.31,23048.17,23001.00,23020.61,478.82619,0,42.420092,46.068729,23055.500,23057.755,23056.4150,23033.915,23271.54,-2.255,17.650,-1
90746,BTCUSDT,2023-02-01 20:26:00,23428.94,23454.25,23423.53,23438.12,550.31760,0,59.530769,58.474188,23398.125,23393.130,23296.5875,23127.500,23487.55,4.995,-15.575,1
90784,BTCUSDT,2023-02-01 21:04:00,23562.92,23575.00,23535.27,23570.40,321.63149,0,56.012647,54.345553,23558.575,23551.000,23541.8575,23461.075,23745.72,7.575,-9.310,1


In [59]:
def use_donchian_channel(df):
    df['upper'], df['lower'], df['mid'] = donchian_channels(df)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['donchian_channel_buy_sell'] = 0

    buy_condition = trading_df.Close >= trading_df.upper
    trading_df.loc[buy_condition, 'donchian_channel_buy_sell'] = 1

    sell_condition = trading_df.Close <= trading_df.lower
    trading_df.loc[sell_condition, 'donchian_channel_buy_sell'] = -1
    
    df.drop(['upper', 'lower', 'mid'], axis = 1, inplace = True)
    return trading_df
    

In [60]:
donchian_test = use_donchian_channel(df)
donchian_test[(donchian_test['donchian_channel_buy_sell'] == 1) | (donchian_test['donchian_channel_buy_sell'] == -1)]


C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\500044658.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['donchian_channel_buy_sell'] = 0


,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell,rsi,prev_rsi,upper,lower,mid,donchian_channel_buy_sell
242,BTCUSDT,2022-12-01 00:02:00,17174.76,17183.76,17157.18,17161.26,226.98035,0,42.840441,48.565894,17222.12,17161.26,17191.690,-1
245,BTCUSDT,2022-12-01 00:05:00,17165.44,17167.28,17152.00,17152.09,174.63185,0,39.819265,45.175142,17203.21,17152.09,17177.650,-1
246,BTCUSDT,2022-12-01 00:06:00,17152.91,17155.63,17139.33,17147.26,227.17760,0,38.069855,39.819265,17203.21,17147.26,17175.235,-1
254,BTCUSDT,2022-12-01 00:14:00,17178.46,17190.54,17177.30,17188.29,164.81992,0,56.969867,52.148319,17188.29,17147.26,17167.775,1
255,BTCUSDT,2022-12-01 00:15:00,17188.85,17196.61,17179.88,17195.25,256.69111,0,59.743856,56.969867,17195.25,17147.26,17171.255,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90939,BTCUSDT,2023-02-01 23:39:00,23706.42,23710.76,23693.66,23697.40,270.47691,0,34.561869,37.079885,23774.05,23697.40,23735.725,-1
90947,BTCUSDT,2023-02-01 23:47:00,23700.04,23700.77,23682.94,23695.34,330.61525,0,36.852020,38.493386,23763.11,23695.34,23729.225,-1
90948,BTCUSDT,2023-02-01 23:48:00,23695.34,23703.68,23690.04,23694.87,143.62819,0,36.683568,36.852020,23763.11,23694.87,23728.990,-1
90957,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,0,54.055783,50.226418,23722.29,23694.87,23708.580,1


In [61]:
def get_data(symbol, interval, lookback):
    data = []
    while data == []:
        data = client.get_historical_klines(symbol,
                                      interval,
                                      str(lookback) + ' minute ago UTC')
    df = pd.DataFrame(data)
    df = df.iloc[:,:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.insert(loc=0, column='Name', value=symbol)
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.Time = pd.to_datetime(df.Time, unit = 'ms')
    # df.set_index('Time', inplace = True)
    
    return df

In [62]:
def get_ta(df, indicator_1 = macd, indicator_2 = rsi):
    if (indicator_1== macd) | (indicator_2== macd):
        df = use_macd(df)
    
    if (indicator_1 == rsi) | (indicator_2 == rsi):
        df = use_rsi(df)
    
    if (indicator_1 == sma) | (indicator_2 == sma):
        df = use_sma(df)
    
    if (indicator_1 == ichimoku_cloud) | (indicator_2 == ichimoku_cloud):
        df = use_ichimoku(df)
    
    if (indicator_1 == donchian_channels) | (indicator_2 == donchian_channels):
        df = use_donchian_channel(df)
    return df

    

In [63]:
test = get_ta(df)
test[(test.iloc[:,-1] !=0) & (test.iloc[:, -2] != 0)]

C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\2050756131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df['rsi_buy_sell'] = 0
C:\Users\nolan\AppData\Local\Temp\ipykernel_17652\2050756131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trading_df.drop(['rsi', 'prev_rsi'], axis = 1, inplace = True)


,Name,Time,Open,High,Low,Close,Volume,macd_buy_sell,rsi_buy_sell
63791,BTCUSDT,2023-01-14 03:11:00,20972.6,20973.22,20834.12,20851.76,1040.45695,-1,1
80768,BTCUSDT,2023-01-25 22:08:00,23708.2,23752.83,23702.66,23722.91,388.74070,-1,-1


In [86]:
def buy(trading_data, buy_type, fiat_amount):  
    trading_data = trading_data[(trading_data.iloc[:,-1] != 0) & (trading_data.iloc[:,-2] != 0)]
    if buy_type == 'all':
        position = False
        coin = 0
        for idx, row in trading_data.iterrows():
            if ((row[-1] == 1) & (row[-2] == 1)) & (position == False):
                position = True
                coin = fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0
                
            elif ((row[-1] == -1) | (row[-2] == -1)) & (position == True):
                fiat_amount = coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    
    if buy_type == 'consec':
        position = False
        fiat_buy = fiat_amount / 4
        coin = 0
        for idx, row in trading_data.iterrows():
            if ((row[-1] == 1) & (row[-2] == 1)) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif ((row[-1] == 1) & (row[-2] == 1)) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif ((row[-1] == -1) | (row[-2] == -1)) & (position == True):
                fiat_amount += coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    return fiat_amount

In [96]:
def backtest(indicator_1, indicator_2, data = df, buy_type='all', fiat_amount = 0 ):
    # while True:
        # try:
        #     fiat_amount = float(input("Please enter your fiat value: "))
        # except ValueError:
        #     #user input not valid
        #     print("please enter a valid number")
        #     continue
        # else:
    trading_data = get_ta(df, indicator_1=indicator_1, indicator_2=indicator_2)
    fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)
    
    return fiat_amount
        #successful input break loop and continue
    # break

    

In [97]:
def trade(token, indicator,stop_event, fiat_amount = 0,  buy_type = 'all'):
    df = get_data(token, Client.KLINE_INTERVAL_1MINUTE, '500')
    df = get_ta(df)
    time.sleep(60)
    if buy_type == 'all':
        position = False
        coin = 0
        while not stop_event.is_set():
            file_mod_time = os.stat('output.txt').st_mtime
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            if (df.iloc[-1,-1] == 1) & (position == False):
                position = True
                coin = fiat_amount / df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Bought {} with {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                fiat_amount = 0
                
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount = coin * df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Sold {} for {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                coin = 0
                position = False
            elif (position == False):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Buy signal..." +"\n" )
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            elif (position == True):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Sell signal..." +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            time.sleep(60)

    if buy_type == 'consec':
        position = False
        coin = 0
        while not stop_event.is_set():
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            fiat_buy = fiat_amount / 4
            if (df.iloc[-1,-1] == 1) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif (df.iloc[-1,-1] == 1) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount += coin * df.iloc[-1, 5]
                print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
            time.sleep(60)

   

In [98]:
#trade(indicator = 'sma', buy_type='consec')

In [99]:
result = {'BTC': [],
               'ETH': [],
               'APE': [],
               'CHZ': [],
               'DOG': [],
               'LIN': [],
               'MAN': [],
               'UNI': [],}

files = ['BTCUSDT_data.csv', 'ETHUSDT_data.csv', 'APEUSDT_data.csv', 'CHZUSDT_data.csv', 'DOGEUSDT_data.csv', 'LINKUSDT_data.csv', 'MANAUSDT_data.csv', 'UNIUSDT_data.csv']

#for file in files:
#    df = pd.read_csv('data/'+file)
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='all'))
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='consec'))
    

In [100]:
#result_macd

Gerald's User Input Prompt to Select the two TA

In [101]:
indicator_functions = {
    'EMA': lambda data: ema(data, 14),
    'MACD': macd,
    'Ichimoku Cloud': ichimoku_cloud,
    'ATR': calculate_atr,
    'Donchian Channels': donchian_channels,
    'RSI': rsi,
    'Bollinger Bands': bollinger_bands,
}

token_list = {
    'BTC': 'BTCUSDT',
    'ETH': 'ETHUSDT',
    'APE': 'APEUSDT',
    'CHZ': 'CHZUSDT',
    'DOG': 'DOGEUSDT',
    'LIN': 'LINKUSDT',
    'MAN': 'MANAUSDT',
    'UNI': 'UNIUSDT',
}


tokens = list(token_list.keys())

def live_trade():
    indicator1 = selected_indicator1.get()
    # indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    with open("output.txt", "w") as file:
            file.write("Initializing..." + "\n")
    
    stop_event = threading.Event()
    t = threading.Thread(target=trade, args=(token_list[token], indicator1.lower(),stop_event, fiat_value, 'all'))
    print("Start Threading")
    t.start()
    new_window = tk.Tk()
    new_window.title("Trading Process")
    
    output_text = tk.Text(new_window)
    output_text.insert(tk.END, 'Initializing...')
    output_text.pack()
    output_text.config(state=tk.DISABLED)
    # trade(token_list[token], indicator=indicator1.lower(), fiat_amount=fiat_value)

    #Multithreading
    update_tk(output_text,new_window)
    new_window.protocol("WM_DELETE_WINDOW", lambda: stop_worker(stop_event,new_window))

    
    new_window.mainloop()

data_files = {
    'BTC': 'data/BTCUSDT_data.csv',
    'ETH': 'data/ETHUSDT_data.csv',
    'APE': 'data/APEUSDT_data.csv',
    'CHZ': 'data/CHZUSDT_data.csv',
    'DOG': 'data/DOGEUSDT_data.csv',
    'LIN': 'data/LINKUSDT_data.csv',
    'MAN': 'data/MANAUSDT_data.csv',
    'UNI': 'data/UNIUSDT_data.csv',
}

dataframes = {}
for token, file in data_files.items():
    df = pd.read_csv(file)
    df.drop(["Unnamed: 0", "index"], axis=1, inplace=True)
    dataframes[token] = df

def submit():
    indicator1 = selected_indicator1.get()
    indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    new_window = tk.Tk()
    new_window.title("Trade Result")
    data = dataframes[token]
    # Calculate the outcomes for the selected indicators
    outcome1 = backtest(indicator_1=indicator1.lower(), indicator_2=indicator2.lower(), data=data, fiat_amount=fiat_value)
    # outcome2 = backtest(indicator=indicator2.lower(), data=data, fiat_amount=fiat_value)
    output_text = tk.Text(new_window, height=10, width=50)
    output_text.pack()
    output_text.insert(tk.END, f"Selected Indicators:\n")
    output_text.insert(tk.END, f"Indicator 1: {indicator1}, Indicator 2: {indicator2}, Outcome: {outcome1}\n")
    # output_text.insert(tk.END, f"Indicator 2: {indicator2}, Outcome: {outcome2}\n")
    output_text.insert(tk.END, f"Fiat Value: {fiat_value}, Token: {token}\n")


def update_tk(output_text, window):
    with open("output.txt", "r") as file:
        content = file.read()
    output_text.config(state=tk.NORMAL)
    output_text.delete('1.0', tk.END)
    output_text.insert(tk.END, content)
    output_text.config(state=tk.DISABLED) 
    print('Screen refreshed')
    window.after(61500, lambda: update_tk(output_text, window))

def stop_worker(stop_event, window):
    stop_event.set()
    window.destroy()





# Create the tkinter window
window = tk.Tk()
window.title("Select Technical Analysis Indicators, Fiat Value, and Token")

# Create the labels and dropdowns for the two indicators
label1 = ttk.Label(window, text="Select Indicator 1:")
label1.grid(column=0, row=0)
selected_indicator1 = tk.StringVar()
dropdown1 = ttk.Combobox(window, textvariable=selected_indicator1, values=list(indicator_functions.keys()))
dropdown1.grid(column=1, row=0)

label2 = ttk.Label(window, text="Select Indicator 2 (Backtest only):")
label2.grid(column=0, row=1)
selected_indicator2 = tk.StringVar()
dropdown2 = ttk.Combobox(window, textvariable=selected_indicator2, values=list(indicator_functions.keys()))
dropdown2.grid(column=1, row=1)

# Create the labels and entry for the fiat value
label3 = ttk.Label(window, text="Enter Fiat Value:")
label3.grid(column=0, row=2)
fiat_value_entry = ttk.Entry(window)
fiat_value_entry.grid(column=1, row=2)

#Create the labels and dropdown for the token
label4 = ttk.Label(window, text="Select Token:")
label4.grid(column=0, row=3)
selected_token = tk.StringVar()
dropdown3 = ttk.Combobox(window, textvariable=selected_token, values=tokens)
dropdown3.grid(column=1, row=3)

#Create a submit button
submit_button = ttk.Button(window, text="Trade", command=live_trade)
submit_button.grid(column=1, row=4)
submit_button = ttk.Button(window, text="Backtest", command=submit)
submit_button.grid(column=0, row=4)

# Start the tkinter main loop
window.mainloop()


In [ ]:
910.599224162299

# Backtest

In [71]:
df = pd.read_csv('./year_data/ETHUSDT_full.csv')

FileNotFoundError: [Errno 2] No such file or directory: './year_data/ETHUSDT_full.csv'

In [ ]:
files = ['BTCUSDT_full.csv', 'ETHUSDT_full.csv', 'APEUSDT_full.csv', 'CHZUSDT_full.csv', 'DOGEUSDT_full.csv', 'LINKUSDT_full.csv', 'MANAUSDT_full.csv', 'UNIUSDT_full.csv']
indicators = [use_macd, use_rsi, use_sma, use_donchian_channel, use_ichimoku]
result_df = pd.DataFrame(index=['BTC','ETH','APE','CHZ','DOG','LIN','MAN','UNI'],columns=['use_macd & use_rsi', 'use_macd & use_sma', 'use_rsi & use_sma'])
for file in files:
    df = pd.read_csv("./year_data/{}".format(file))
    for i in range(len(indicators)):
        for j in range(i + 1,len(indicators)):
            print('Testing:{} & {}'.format(indicators[i].__name__, indicators[j].__name__))
            trading_frame_1 = indicators[i](df)
            trading_frame_2 = indicators[j](df)
            trading_data = df.copy()
            trading_data['buy_sell_1'] = trading_frame_1.iloc[:,-1]
            trading_data['buy_sell_2'] = trading_frame_2.iloc[:,-1]
            
            position = False
            coin = 0
            fiat_amount = 1000
            for idx, row in trading_data.iterrows():
                if (row['buy_sell_1'] == 1) & (row['buy_sell_2'] == 1) & (position == False):
                    position = True
                    coin = fiat_amount / row['Close']
                    print("Bought {} with {}".format(coin, fiat_amount))
                    fiat_amount = 0
                    
                elif ((row['buy_sell_1'] == -1) | (row['buy_sell_2'] == -1)) & (position == True):
                    fiat_amount = coin * row['Close']
                    print("Sold {} for {}".format(coin, fiat_amount))
                    coin = 0
                    position = False
            fiat_amount += coin * trading_data.iloc[-1, 5]
            coin = 0
            
            result_df.loc[file[0:3], '{} & {}'.format(indicators[i].__name__, indicators[j].__name__) ] = fiat_amount



Testing:use_macd & use_rsi


KeyboardInterrupt: 

In [ ]:
result_df